<a href="https://colab.research.google.com/github/Mwadz/week-8-ip/blob/main/Week_8_IP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd